<a href="https://colab.research.google.com/github/madhura2024/myAI_MODELS/blob/main/dressrecs_apriori_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

APRIORI ASSOCIATION RULES

In [12]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
cloth = pd.read_csv("Fashion_Retail_Sales.csv")
cloth.head(5)

,Customer Reference ID,Item Purchased,Purchase Amount (USD),Date Purchase,Review Rating,Payment Method
0,4018,Handbag,4619.0,05-02-2023,NaN,Credit Card
1,4115,Tunic,2456.0,11-07-2023,2.0,Credit Card
2,4019,Tank Top,2102.0,23-03-2023,4.1,Cash
3,4097,Leggings,3126.0,15-03-2023,3.2,Cash
4,3997,Wallet,3003.0,27-11-2022,4.7,Cash


clean data

In [13]:
cloth = cloth.dropna(subset=['Item Purchased', 'Customer Reference ID']).drop_duplicates()
transactions = pd.get_dummies(cloth['Item Purchased'])
transactions = transactions.groupby(cloth['Customer Reference ID']).sum()

1. transactions has to be > 1 ********* rule in apriori cannot be null so just check .
2.use_cols used so that it shows jeans , shirts instead of 0 ,1
3. use metric lift , min threshold so that model is responsive to weak data

In [14]:
from mlxtend.frequent_patterns import apriori, association_rules

# Convert to 0/1
transactions = transactions.clip(upper=1).astype(bool)

# Frequent itemsets
frequent_itemsets = apriori(transactions, min_support=0.1, use_colnames=True)

# Association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# print(rules.head())


In [15]:
def recommend(item):
    for index, row in rules.iterrows():
        if item in row['antecedents']:   # check antecedents
            return list(row['consequents'])[:5]
    return None

import warnings
warnings.filterwarnings("ignore")

# choose a valid item from your rules like 'T-shirt' or 'Skirt'
recommendeditems = recommend('Skirt')

if recommendeditems is not None:
    print("Recommended items for T-shirt:", recommendeditems)
else:
    print("No recommendations found.")


Recommended items for T-shirt: ['Belt']


Why Kurti worked earlier (with low support)

When you had min_support=0.001, Apriori was generating rules even for rare combinations (things that happened very few times).
That’s why "Kurti" appeared — but those rules were statistically weak (maybe only 1 or 2 co-occurrences).

So technically:

It was not wrong, but it was very weak/noisy.

Apriori included it only because you allowed very low support.

eclat

In [18]:
from mlxtend.frequent_patterns import fpgrowth, association_rules
import warnings
warnings.filterwarnings("ignore")

transactions = transactions.clip(upper=1).astype(bool)
frequent_itemsets = fpgrowth(transactions, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

def recommend(item):
    for index, row in rules.iterrows():
        if item in row['antecedents']:
            return list(row['consequents'])[:5]
    return None

recommendeditems = recommend('Skirt')
if recommendeditems is not None:
    print("Recommended items for Skirt:", recommendeditems)
else:
    print("No recommendations found.")


Recommended items for Skirt: ['Tank Top']
